# Ensemble Notebook (Catalyst)

### Experiments

* [x] Efficientnet b5 **PL:0.652**
* [x] FP 16，refer to this [catalyst tutorial](https://github.com/catalyst-team/catalyst/blob/master/examples/notebooks/segmentation-tutorial.ipynb)
    * The model will have gradient overflow after 5th epoch, everything else is okay
* [x] Saving & Loading from JIT **PL:0.655**
* [x] Ensemble
* [x] 384x576
* [x] polygon convex
* [ ] Test the funnel network again
* [ ] Ranger optimizer 
    * [ ] RADAM
    * [ ] Look Ahead

### Installing Apex for FP16

```shell
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
is_fp16_used = True
```

### Other Installations

```
pip install catalyst
pip install pretrainedmodels
pip install git+https://github.com/qubvel/segmentation_models.pytorch
pip install pip pytorch-toolbelt
pip install torchvision==0.4
```

Our starter kernel is from [this open kernel](https://www.kaggle.com/artgor/segmentation-in-pytorch-using-convenient-tools)

## ======== Below is from the original notebook ===========

## General information

In this kernel I work with the data from Understanding Clouds from Satellite Images competition.
```
Shallow clouds play a huge role in determining the Earth's climate. They’re also difficult to understand and to represent in climate models. By classifying different types of cloud organization, researchers at Max Planck hope to improve our physical understanding of these clouds, which in turn will help us build better climate models.
```

So in this competition we are tasked with multiclass segmentation task: finding 4 different cloud patterns in the images. On the other hand, we make predictions for each pair of image and label separately, so this could be treated as 4 binary segmentation tasks.
It is important to notice that images (and masks) are `1400 x 2100`, but predicted masks should be `350 x 525`.

In this kernel I'll use (or will use in next versions) the following notable libraries:
- [albumentations](https://github.com/albu/albumentations): this is a great library for image augmentation which makes it easier and more convenient
- [catalyst](https://github.com/catalyst-team/catalyst): this is a great library which makes using PyTorch easier, helps with reprodicibility and contains a lot of useful utils
- [segmentation_models_pytorch](https://github.com/qubvel/segmentation_models.pytorch): this is a great library with convenient wrappers for models, losses and other useful things
- [pytorch-toolbelt](https://github.com/BloodAxe/pytorch-toolbelt): this is a great library with many useful shortcuts for building pytorch models


UPD: Version 35 - changed calculation of optimal threshold and min size
![](https://i.imgur.com/EOvz5kd.png)

## Importing libraries

In [1]:
import os
import cv2
import collections
import time 
import tqdm
from PIL import Image
from functools import partial
train_on_gpu = True

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
from albumentations import pytorch as AT

from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

import segmentation_models_pytorch as smp

## Helper functions and classes

#### Configs and Hyper-Params

In [2]:
from utils_ucsi import *

In [3]:
TRAIN = False # Do we have training?
num_epochs = 40 # How many epochs are we going to train?

FP16 = False # Do we use half precision?
fp16_params = dict(opt_level = "O2") if FP16 else None

LOAD = False # Do we load a trained weights at the beginning
LOAD_PATH = "cata-eff-b5.pth" # The model weight path, if we load a trained weights at the begining

ENCODER = 'efficientnet-b5' # Encoder model name
ENCODER_WEIGHTS = 'imagenet' # Encoder pretrained weights
DEVICE = 'cuda' 

ACTIVATION = None

MIN_SIZE_RANGE = 3
MIN_SIZE = [0, 100, 1200, 5000,8000, 10000][-MIN_SIZE_RANGE:]

INPUT_SIZE = (384,576)

In [4]:
MODEL_PRED = True
# Ensemble Model Path List

MODEL_PATHS = [
    {"path":"pth/fpn_se_resnext.pth","encoder":"se_resnext101_32x4d"},
#     {"path":"pth/senet154_384x576.pth","encoder":"senet154"},
]


# INFER_BS = 28

## Encoders
Encoder backbone of fpn/unet
#### VGG
vgg11, vgg13, vgg16, vgg19, vgg11bn, vgg13bn, vgg16bn, vgg19bn,
#### DenseNet
densenet121, densenet169, densenet201, densenet161, dpn68, dpn98, dpn131,
inceptionresnetv2,
#### ResNet
resnet18, resnet34, resnet50, resnet101, resnet152,
resnext50_32x4d, resnext101_32x8d,
#### Se ResNet
se_resnet50, se_resnet101, se_resnet152,
#### Se ResNext
se_resnext50_32x4d, se_resnext101_32x4d,
#### Se Net
senet154,
#### Efficient Net
efficientnet-b0, efficientnet-b1, efficientnet-b2, efficientnet-b3, efficientnet-b4, efficientnet-b5, efficientnet-b6, efficientnet-b7

## Data overview

Let's have a look at the data first.

In [5]:
from pathlib import Path
HOME = Path(os.environ["HOME"])

path = HOME/'ucsi'
# os.listdir(path)

We have folders with train and test images, file with train image ids and masks and sample submission.

In [6]:
train = pd.read_csv(f'{path}/train.csv')
sub = pd.read_csv(f'{path}/sample_submission.csv')

In [7]:
train.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [8]:
n_train = len(os.listdir(f'{path}/train_images'))
n_test = len(os.listdir(f'{path}/test_images'))
print(f'There are {n_train} images in train dataset')
print(f'There are {n_test} images in test dataset')

There are 5546 images in train dataset
There are 3698 images in test dataset


In [9]:
train['Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()

Flower    5546
Sugar     5546
Fish      5546
Gravel    5546
Name: Image_Label, dtype: int64

So we have ~5.5k images in train dataset and they can have up to 4 masks: Fish, Flower, Gravel and Sugar.

In [10]:
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[1]).value_counts()

Sugar     3751
Gravel    2939
Fish      2781
Flower    2365
Name: Image_Label, dtype: int64

In [11]:
train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().value_counts()

2    2372
3    1560
1    1348
4     266
Name: Image_Label, dtype: int64

But there are a lot of empty masks. In fact only 266 images have all four masks. It is important to remember this.

In [12]:
train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])


sub['label'] = sub['Image_Label'].apply(lambda x: x.split('_')[1])
sub['im_id'] = sub['Image_Label'].apply(lambda x: x.split('_')[0])

Let's have a look at the images and the masks.

In [13]:
#fig = plt.figure(figsize=(25, 16))
#for j, im_id in enumerate(np.random.choice(train['im_id'].unique(), 4)):
#    for i, (idx, row) in enumerate(train.loc[train['im_id'] == im_id].iterrows()):
#        ax = fig.add_subplot(5, 4, j * 4 + i + 1, xticks=[], yticks=[])
#        im = Image.open(f"{path}/train_images/{row['Image_Label'].split('_')[0]}")
#        plt.imshow(im)
#       mask_rle = row['EncodedPixels']
#        try: # label might not be there!
#            mask = rle_decode(mask_rle)
#        except:
#            mask = np.zeros((1400, 2100))
#        plt.imshow(mask, alpha=0.5, cmap='gray')
#        ax.set_title(f"Image: {row['Image_Label'].split('_')[0]}. Label: {row['label']}")

We can see that masks can overlap. Also we can see that clouds are really similar to fish, flower and so on. Another important point: masks are often quite big and can have seemingly empty areas.

## Preparing data for modelling

At first, let's create a list of unique image ids and the count of masks for images. This will allow us to make a stratified split based on this count.

In [14]:
id_mask_count = train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().\
reset_index().rename(columns={'index': 'img_id', 'Image_Label': 'count'})
train_ids, valid_ids = train_test_split(id_mask_count['img_id'].values, random_state=42, stratify=id_mask_count['count'], test_size=0.1)
test_ids = sub['Image_Label'].apply(lambda x: x.split('_')[0]).drop_duplicates().values

## Exploring augmentations with albumentations

One of important things while working with images is choosing good augmentations. There are a lot of them, let's have a look at augmentations from albumentations!

In [15]:
#image_name = '8242ba0.jpg'
#image = get_img(image_name)
#mask = make_mask(train, image_name)

#visualize(image, mask)

# This is how original image and its masks look like. Let's try adding some augmentations

#plot_with_augmentation(image, mask, albu.HorizontalFlip(p=1))

#plot_with_augmentation(image, mask, albu.VerticalFlip(p=1))

#plot_with_augmentation(image, mask, albu.RandomRotate90(p=1))

#plot_with_augmentation(image, mask, albu.ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03))

#plot_with_augmentation(image, mask, albu.GridDistortion(p=1))

#plot_with_augmentation(image, mask, albu.OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5))

## Setting up data for training in Catalyst

Now we define model and training parameters

In [16]:
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=4, 
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [17]:
num_workers = 0
bs = 10 if FP16 else 5
train_dataset = CloudDataset(df=train, datatype='train', img_ids=train_ids, transforms = get_training_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))
valid_dataset = CloudDataset(df=train, datatype='valid', img_ids=valid_ids, transforms = get_validation_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))

train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

loaders = {
    "train": train_loader,
    "valid": valid_loader
}

/home/b2ray2c/anaconda3/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2875: UserWarning:

Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().



In [18]:

logdir = "./logs/segmentation"

# model, criterion, optimizer
optimizer = torch.optim.Adam([
    {'params': model.decoder.parameters(), 'lr': 1e-3}, 
    {'params': model.encoder.parameters(), 'lr': 5e-4},  # Pretrained section of the model using smaller lr
], 
    weight_decay=3e-4)
scheduler = ReduceLROnPlateau(optimizer, factor=0.25, patience=2)
criterion = smp.utils.losses.BCEDiceLoss(eps=1.)
runner = SupervisedRunner()

In [19]:
if TRAIN:
    model = model.cuda()

In [20]:
if LOAD:
    model.load_state_dict(torch.load(LOAD_PATH))

## Model training

In [24]:
if TRAIN:
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        loaders=loaders,
        callbacks=[DiceCallback(), EarlyStoppingCallback(patience=5, min_delta=0.001)],
        logdir=logdir,
        num_epochs=num_epochs,
        fp16=fp16_params,
        verbose=True
    )

    torch.save(runner.model.state_dict(),'pth/final_%s.pth'%(datetime.now().strftime("%m%d_%H%M%S")))

    
    utils.plot_metrics(
        logdir=logdir, 
        # specify which metrics we want to plot
        metrics=["loss", "dice", 'lr', '_base/lr']
    )

In [25]:
if TRAIN:
    print("="*100)
    print("Saving to JIT model")
    runner.model = runner.model.eval()
    traced = torch.jit.trace(runner.model, torch.rand(2,3,*INPUT_SIZE).cuda())
    traced.save("jit/jit-cata-eff-b5-v2.pth")

## Exploring predictions
Let's make predictions on validation dataset.

At first we need to optimize thresholds 

In [26]:
infer_cb = []

In [27]:
class ensModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = models
    
    def __call__(self, x):
        res = []
        x = x.cuda()
        with torch.no_grad():
            for m in self.models:
                res.append(m(x))
        res = torch.stack(res)
        return torch.mean(res, dim=0)

In [28]:
if MODEL_PRED:
    print("Loading ensemble models")
    print("="*70)
    models = list(loadModel(path = p["path"],encoder = p["encoder"]) for p in MODEL_PATHS)
    if torch.cuda.is_available():
        models = list(m.cuda() for m in models)
    model = ensModel(models)
else:
    infer_cb.append(CheckpointCallback(resume=f"{logdir}/checkpoints/best.pth"),)
infer_cb.append(InferCallback())

Loading ensemble models
loading se_resnext101_32x4d from path 'pth/fpn_se_resnext.pth'


In [29]:
encoded_pixels = []
loaders = {"infer": valid_loader}
runner.infer(
    model=model,
    loaders=loaders,
    callbacks=infer_cb,
)
valid_masks = []
probabilities = np.zeros((2220, 350, 525))
for i, (batch, output) in enumerate(tqdm.tqdm(zip(
        valid_dataset, runner.callbacks[0].predictions["logits"]))):
    image, mask = batch
    for m in mask:
        if m.shape != (350, 525):
            m = cv2.resize(m, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
        valid_masks.append(m)

    for j, probability in enumerate(output):
        if probability.shape != (350, 525):
            probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
        probabilities[i * 4 + j, :, :] = probability

555it [00:59,  9.34it/s]


## Find optimal values

First of all, my thanks to @samusram for finding a mistake in my validation
https://www.kaggle.com/c/understanding_cloud_organization/discussion/107711#622412

And now I find optimal values separately for each class.

In [30]:
class_params = {}
for class_id in range(4):
    print(class_id)
    attempts = []
    for t in range(0, 100, 5):
        t /= 100
        for ms in MIN_SIZE:
            masks = []
            for i in range(class_id, len(probabilities), 4):
                probability = probabilities[i]
                predict, num_predict = post_process(sigmoid(probability), t, ms)
                masks.append(predict)

            d = []
            for i, j in zip(masks, valid_masks[class_id::4]):
                if (i.sum() == 0) & (j.sum() == 0):
                    d.append(1)
                else:
                    d.append(dice(i, j))

            attempts.append((t, ms, np.mean(d)))

    attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])
    attempts_df = attempts_df.sort_values('dice', ascending=False)
    print(attempts_df.head())
    best_threshold = attempts_df['threshold'].values[0]
    best_size = attempts_df['size'].values[0]
    
    class_params[class_id] = (best_threshold, best_size)

0
    threshold   size      dice
23       0.35  10000  0.606897
26       0.40  10000  0.605114
29       0.45  10000  0.604266
25       0.40   8000  0.603501
28       0.45   8000  0.603165
1
    threshold   size      dice
41       0.65  10000  0.716129
52       0.85   8000  0.714070
53       0.85  10000  0.713647
38       0.60  10000  0.713396
44       0.70  10000  0.713016
2
    threshold   size      dice
32       0.50  10000  0.590334
29       0.45  10000  0.587162
47       0.75  10000  0.587067
34       0.55   8000  0.586779
35       0.55  10000  0.586186
3
    threshold   size      dice
32       0.50  10000  0.596285
29       0.45  10000  0.594540
28       0.45   8000  0.593921
31       0.50   8000  0.592836
26       0.40  10000  0.592549


In [31]:
print(class_params)

{0: (0.35, 10000), 1: (0.65, 10000), 2: (0.5, 10000), 3: (0.5, 10000)}


In [32]:
#sns.lineplot(x='threshold', y='dice', hue='size', data=attempts_df);
#plt.title('Threshold and min size vs dice for one of the classes');

Now let's have a look at our masks.

In [33]:
#for i, (input, output) in enumerate(zip(
#        valid_dataset, runner.callbacks[0].predictions["logits"])):
#    image, mask = input
#        
#    image_vis = image.transpose(1, 2, 0)
#    mask = mask.astype('uint8').transpose(1, 2, 0)
#    pr_mask = np.zeros((350, 525, 4))
#    for j in range(4):
#        probability = cv2.resize(output.transpose(1, 2, 0)[:, :, j], dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
#        pr_mask[:, :, j], _ = post_process(sigmoid(probability), class_params[j][0], class_params[j][1])
    #pr_mask = (sigmoid(output) > best_threshold).astype('uint8').transpose(1, 2, 0)
    
        
#    visualize_with_raw(image=image_vis, mask=pr_mask, original_image=image_vis, original_mask=mask, raw_image=image_vis, raw_mask=output.transpose(1, 2, 0))
    
#    if i >= 2:
#        break

## Predicting

In [34]:
import gc
torch.cuda.empty_cache()
gc.collect()

80

In [35]:
test_dataset = CloudDataset(df=sub, datatype='test', img_ids=test_ids, transforms = get_validation_augmentation(), preprocessing=get_preprocessing(preprocessing_fn))
test_loader = DataLoader(test_dataset, batch_size=36, shuffle=False, num_workers=0)

loaders = {"test": test_loader}

In [36]:
encoded_pixels = []
image_id = 0
for i, test_batch in enumerate(tqdm.tqdm(loaders['test'])):
    runner_out = runner.predict_batch({"features": test_batch[0].cuda()})['logits']
    for i, batch in enumerate(runner_out):
        for probability in batch:
            
            probability = probability.cpu().detach().numpy()
            if probability.shape != (350, 525):
                probability = cv2.resize(probability, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
            predict, num_predict = post_process(sigmoid(probability), class_params[image_id % 4][0], class_params[image_id % 4][1])
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                r = mask2rle(predict)
                encoded_pixels.append(r)
            image_id += 1

100%|██████████| 103/103 [15:40<00:00,  9.13s/it]


In [37]:
from datetime import datetime

In [38]:
sub['EncodedPixels'] = encoded_pixels
sub.to_csv('%s_submission.csv'%(datetime.now().strftime("%m%d_%H%M%S")), columns=['Image_Label', 'EncodedPixels'], index=False)